**Name - Amit George**

**Task 3 - Social Distancing detector** 

**Grip@ The Spark foundation network**

**Objective- To detect social distancing between people within a certain crowd**

**Load the libraries**

In [6]:
import os
import matplotlib.pyplot as plt
import cv2
from natsort import natsorted
from PIL import Image
import glob
import shutil
import torchvision
from IPython.display import HTML
from base64 import b64encode
import torch
import numpy as np
from google.colab.patches import cv2_imshow

**Loading the Model for detecting Person's**

In [7]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5x',
                       pretrained=True, verbose=False)
#model.cuda('cuda:0');

requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-6-18 Python-3.7.13 torch-1.11.0+cu113 CPU

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
# loading video-files from drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip '/content/sample_data/ezgif-4-a819b38007-jpg.zip' -d '/content/video-files'

In [99]:
#input_dir='/content/drive/MyDrive/video-files'
input_dir= '/content/video-files'
images = natsorted(glob.glob(os.path.join(input_dir, '*.jpg')))

**Obtaining the bounding box coordinates of detected person's from the images** 

In [100]:
dataset=[]
for img in images:
 image=cv2.imread(img)
 image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
 results = model([image[:, :, ::-1]])  # Pass the frame through the model and get the boxes
 results=results.xyxy[0].numpy()
 results=results[results[:,5]==0] # only people
 data=results[:,:4] # take the first 4 bounding box corrdiates of pixels.
 dataset.append(data)

**Calculate the centre distance between two persons's in the image**

In [107]:
def center_distance(xyxy1, xyxy2):
    #Calculate the distance of the centers of the boxes.
    a, b, c, d = xyxy1
    x1 = int(np.mean([a, c]))
    y1 = int(np.mean([b, d]))

    e, f, g, h = xyxy2
    x2 = int(np.mean([e, g]))
    y2 = int(np.mean([f, h]))
    
    dist = np.linalg.norm([x1 - x2, y1 - y2])
    return dist, x1, y1, x2, y2

**Calculate the social distance between people in the image, the red box means the person is close to the other person within 100 units and green box when far away from each other**

In [145]:
image_list=[]
for objects,img in zip(dataset,images):
 image=cv2.imread(img)
 image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
 for i, (x1, y1, x2, y2) in enumerate(objects):
        color = (0, 255, 0)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
 image_list.append(image)


In [ ]:
image_list2=[]
distance=80 # or 60
for objects,image in zip(dataset,image_list):
  colors = ['green']*len(objects)
  for i in range(len(objects)):
        for j in range(len(objects)):
            if i!=j:
             print(i,j)
             # Calculate distance of the centers
             dist, cx1, cy1, cx2, cy2 = center_distance(objects[i], objects[j])
             #distances.append(dist)
             if dist < distance:
                 x1,y1,x2,y2=objects[i]
                 #distances.append(dist)
                 # If dist < distance, boxes are red and a line is drawn
                 color = (255, 0, 0)
                 image = cv2.line(image, (cx1, cy1), (cx2, cy2), (0, 0, 255), 2)
                 image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
                 break;
  image_list2.append(image)              
                


In [154]:
!mkdir '/content/drive/MyDrive/social_distancing'

**Save the object detection images to be later converted to video**

In [155]:
from PIL import Image
for i,j in zip(image_list2,images):
  im = Image.fromarray(i)
  im.save("/content/drive/MyDrive/social_distancing/{}.jpeg".format(os.path.basename(j)))

In [ ]:
# zip file for the data of images to be converted to video
!zip -r '/content/social_distancing.zip' '/content/drive/MyDrive/social_distancing' 

**Display the video of social distancing**

In [ ]:
video_path='/content/sample_data/social_distance.mp4'
mp4 = open(video_path, "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=400 controls><source src="{data_url}" type="video/mp4">
</video>""")